# Group By, Having & Count
Obtenez des informations plus intéressantes directement à partir de vos requêtes SQL.

## Introduction
Maintenant que vous savez comment sélectionner les données brutes, vous êtes prêt à apprendre à regrouper vos données et à compter les éléments au sein de ces groupes. Cela peut vous aider à répondre à des questions comme :

- Combien de chaque type de fruit notre magasin a-t-il vendu ?
- Combien d'espèces d'animaux le cabinet vétérinaire a-t-il traitées ?

Pour cela, vous allez apprendre trois nouvelles techniques : **GROUP BY, HAVING et COUNT()**. Encore une fois, nous utiliserons cette table fictive d'informations sur les animaux de compagnie.

---
### COUNT()
COUNT(), comme vous pouvez l’imaginer d'après son nom, renvoie un décompte des éléments. Si vous lui passez le nom d'une colonne, elle renverra le nombre d'entrées dans cette colonne.

Par exemple, si nous sélectionnons le COUNT() de la colonne ID dans la table des animaux de compagnie, cela renverra 4, car il y a 4 ID dans la table.

**COUNT()** est un exemple de fonction d'agrégation, qui prend plusieurs valeurs et en renvoie une seule. (D'autres exemples de fonctions d'agrégation incluent **SUM(), AVG(), MIN(), et MAX()**). Comme vous pouvez le remarquer sur l’image ci-dessus, les fonctions d'agrégation introduisent des noms de colonnes étranges (comme f0__). Plus tard dans ce tutoriel, vous apprendrez comment changer ce nom en quelque chose de plus descriptif.

---

### GROUP BY
GROUP BY prend le nom d'une ou plusieurs colonnes et regroupe toutes les lignes ayant la même valeur dans cette colonne en un seul groupe lorsque vous appliquez des fonctions d'agrégation comme **COUNT()**.

Par exemple, supposons que nous voulons savoir combien de chaque type d'animal nous avons dans la table des animaux de compagnie. Nous pouvons utiliser GROUP BY pour regrouper les lignes qui ont la même valeur dans la colonne "Animal", tout en utilisant COUNT() pour savoir combien d'ID nous avons dans chaque groupe.

Cela renverra une table avec trois lignes (une pour chaque animal distinct). Nous pouvons voir que la table des animaux de compagnie contient 1 lapin, 1 chien et 2 chats.

---

### GROUP BY ... HAVING
HAVING est utilisé en combinaison avec GROUP BY pour ignorer les groupes qui ne répondent pas à certains critères.

Par exemple, cette requête n'inclura que les groupes ayant plus d'un ID.

Comme un seul groupe remplit le critère spécifié, la requête renverra une table avec une seule ligne.

---

#### Exemple : Quels commentaires de Hacker News ont généré le plus de discussions ?
Prêt à voir un exemple avec un jeu de données réel ? Le jeu de données Hacker News contient des informations sur des histoires et des commentaires du site de réseautage social Hacker News.

Nous allons travailler avec la table complète et commencer par afficher les premières lignes. (Nous avons caché le code correspondant. Pour y jeter un œil, cliquez sur le bouton "Code" ci-dessous.)

In [4]:
from google.cloud import bigquery

# Créer un objet "Client"
client = bigquery.Client()

# Construire une référence vers le jeu de données "hacker_news"
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# Requête API - récupérer le jeu de données
dataset = client.get_dataset(dataset_ref)

# Construire une référence vers la table "full"
table_ref = dataset_ref.table("full")

# Requête API - récupérer la table
table = client.get_table(table_ref)

# Aperçu des cinq premières lignes de la table
client.list_rows(table, max_results=5).to_dataframe()


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,None,True,Adoum_Tech,2,1713995025,2024-04-24 21:43:45+00:00,story,40150086,<NA>,<NA>,<NA>,<NA>
1,None,None,None,True,belter,2,1713995286,2024-04-24 21:48:06+00:00,story,40150135,<NA>,<NA>,<NA>,<NA>
2,None,None,None,True,Rinzler89,1,1713995678,2024-04-24 21:54:38+00:00,story,40150207,<NA>,<NA>,<NA>,<NA>
3,None,None,None,True,stockstobuynow,1,1713995704,2024-04-24 21:55:04+00:00,story,40150212,<NA>,<NA>,<NA>,<NA>
4,None,None,None,True,FLMAN407,1,1713995772,2024-04-24 21:56:12+00:00,story,40150229,<NA>,<NA>,<NA>,<NA>


Maintenant, **utilisons cette table pour voir quels commentaires ont généré le plus de réponses**. Puisque :

- La colonne **parent** indique le commentaire auquel il a été répondu, et
- La colonne **id** contient l'ID unique pour chaque commentaire,

Nous pouvons utiliser **GROUP BY** sur la colonne parent et COUNT() sur la colonne id pour savoir combien de commentaires ont été faits en réponse à un commentaire spécifique. (Cela peut ne pas avoir de sens immédiatement – prenez votre temps pour vous assurer que tout est clair !)

De plus, comme nous nous intéressons uniquement aux commentaires populaires, nous regarderons ceux ayant plus de dix réponses. Nous ne retournerons donc que les groupes ayant plus de dix ID.

In [7]:
# Requête pour sélectionner les commentaires ayant reçu plus de 10 réponses
query_popular = """
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.full`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """
# Configurer la requête (annuler la requête si elle utilise trop de quota, avec la limite définie à 10 Go)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_popular, job_config=safe_config)

# Requête API - exécuter la requête et convertir les résultats en DataFrame pandas
popular_comments = query_job.to_dataframe()

# Afficher les cinq premières lignes du DataFrame
popular_comments.head()


,parent,f0_
0,40115482,109
1,40197346,66
2,40227208,88
3,40230790,102
4,461296,42


Chaque ligne du DataFrame **popular_comments** correspond à un commentaire ayant reçu plus de dix réponses. Par exemple, le commentaire avec l'ID 801208 a reçu 56 réponses.

---

## Alias et autres améliorations
Quelques astuces pour rendre vos requêtes encore meilleures :
- La colonne résultant de COUNT(id) a été appelée **f0__**. Ce n'est pas un nom très descriptif. Vous pouvez changer ce nom en ajoutant **AS NumPosts** après avoir spécifié l'agrégation. Cela s'appelle **l'aliasing**, et nous en parlerons plus en détail dans un prochain cours.
- Si vous n'êtes jamais sûr de ce que vous devez mettre à l'intérieur de la fonction **COUNT()**, *vous pouvez faire COUNT(1) pour compter les lignes dans chaque groupe*. La plupart des gens trouvent cela particulièrement lisible, car nous savons que cela ne se concentre pas sur d'autres colonnes. Cela permet aussi de scanner moins de données que si des noms de colonnes sont fournis (ce qui rend la requête plus rapide et utilise moins de votre quota d'accès aux données).

En utilisant ces astuces, nous pouvons réécrire notre requête :

In [10]:
# Version améliorée de la requête précédente, maintenant avec aliasing et meilleure lisibilité
query_improved = """
                 SELECT parent, COUNT(1) AS NumPosts
                 FROM `bigquery-public-data.hacker_news.full`
                 GROUP BY parent
                 HAVING COUNT(1) > 10
                 """
# Configurer la requête (annuler la requête si elle utilise trop de quota, avec la limite définie à 10 Go)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_improved, job_config=safe_config)

# Requête API - exécuter la requête et convertir les résultats en DataFrame pandas
improved_comments = query_job.to_dataframe()

# Afficher les cinq premières lignes du DataFrame
improved_comments.head()


,parent,NumPosts
0,4596375,268
1,4700160,55
2,4721751,48
3,250704,118
4,4252955,60


---

### Remarque sur l'utilisation de GROUP BY
Notez qu'il est inutile d'utiliser GROUP BY sans une fonction d'agrégation (comme COUNT()). De même, si vous avez une clause GROUP BY, toutes les variables doivent être soit passées à une fonction d'agrégation, soit à la commande GROUP BY.

Voici un exemple de requête correcte :

```
query_good = """
             SELECT parent, COUNT(id)
             FROM `bigquery-public-data.hacker_news.full`
             GROUP BY parent
             """
```

Ici, parent a été passé à la commande GROUP BY, et id a été passé à une fonction d'agrégation (COUNT(id)).

Et voici une requête **incorrecte**, car la colonne by n'est pas passée à une fonction d'agrégation ou à une clause GROUP BY :

```
query_bad = """
            SELECT `by` AS author, parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.full`
            GROUP BY parent
            """
```

Si vous faites cette erreur, vous obtiendrez le message d'erreur suivant : "SELECT list expression references column (nom de la colonne) which is neither grouped nor aggregated."

Vous remarquerez que la colonne by est entourée de backticks. Cela est dû au fait que BY est un mot réservé utilisé dans des clauses comme GROUP BY. Dans BigQuery, les mots réservés utilisés comme identifiants doivent être encadrés de backticks pour éviter une erreur. Nous avons aussi rendu la référence à cette colonne plus lisible en ajoutant un alias pour la renommer en author.

##### À vous de jouer
Ces agrégations vous permettent d'écrire des requêtes beaucoup plus intéressantes. Essayez par vous-même avec ces exercices de codage.

---


# EXERCICE
#### Introduction
Les requêtes avec **GROUP BY** peuvent être puissantes. Il y a de nombreux petits détails qui peuvent vous induire en erreur (comme l'ordre des clauses), mais cela deviendra naturel après quelques essais. Ici, vous écrirez des requêtes utilisant GROUP BY pour répondre à des questions à partir du dataset Hacker News.

La cellule de code ci-dessous récupère la table complète du dataset hacker_news. Nous prévisualisons également les cinq premières lignes de la table.

In [3]:
from google.cloud import bigquery

# Créer un objet "Client"
client = bigquery.Client()

# Construire une référence au dataset "hacker_news"
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# Requête API - récupérer le dataset
dataset = client.get_dataset(dataset_ref)

# Construire une référence à la table "full"
table_ref = dataset_ref.table("full")

# Requête API - récupérer la table
table_exo = client.get_table(table_ref)

# Prévisualiser les cinq premières lignes de la table
client.list_rows(table_exo, max_results=5).to_dataframe()


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,None,True,Adoum_Tech,2,1713995025,2024-04-24 21:43:45+00:00,story,40150086,<NA>,<NA>,<NA>,<NA>
1,None,None,None,True,belter,2,1713995286,2024-04-24 21:48:06+00:00,story,40150135,<NA>,<NA>,<NA>,<NA>
2,None,None,None,True,Rinzler89,1,1713995678,2024-04-24 21:54:38+00:00,story,40150207,<NA>,<NA>,<NA>,<NA>
3,None,None,None,True,stockstobuynow,1,1713995704,2024-04-24 21:55:04+00:00,story,40150212,<NA>,<NA>,<NA>,<NA>
4,None,None,None,True,FLMAN407,1,1713995772,2024-04-24 21:56:12+00:00,story,40150229,<NA>,<NA>,<NA>,<NA>


## Question 1: Commentateurs prolifiques
Hacker News souhaite récompenser toutes **les personnes ayant écrit plus de 10 000 publications**. Écrivez une requête qui retourne tous *les auteurs ayant plus de 10 000 publications, ainsi que leur nombre total de publications*. Nommez la colonne contenant le nombre de publications NumPosts.


In [6]:
prolific_commenters_query = """
                            SELECT `by` AS author, COUNT(1) AS NumPosts
                            FROM `bigquery-public-data.hacker_news.full`
                            GROUP BY author
                            HAVING COUNT(1) > 10000
                            """

# Configurer la requête (annuler l'exécution si elle dépasse le quota fixé à 1 Go)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)  
query_job = client.query(prolific_commenters_query, job_config=safe_config)

# Exécuter la requête via l'API et retourner un DataFrame pandas
prolific_commenters = query_job.to_dataframe()

# Afficher les premières lignes des résultats
print(prolific_commenters.head())

     author  NumPosts
0      None   1310799
1   Someone     13133
2    btilly     10867
3   aidenn0     12432
4  IshKebab     10839


## Question 2: Commentaires supprimés
Combien de commentaires ont été supprimés ? (Si un commentaire a été supprimé, la colonne deleted dans la table aura la valeur **True**.)

In [9]:
# Query to count deleted comments
deleted_comments_query = """
                          SELECT COUNT(*) AS num_deleted_posts
                          FROM `bigquery-public-data.hacker_news.full`
                          WHERE deleted = TRUE
                          """

# Set up the query (limit resource usage)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(deleted_comments_query, job_config=safe_config)

# API request - run the query and get the result
num_deleted_posts = query_job.to_dataframe().iloc[0, 0]  
# iloc[0, 0] → Extrait la valeur du résultat pour l'affecter à num_deleted_posts.

# View the result
print(num_deleted_posts)


0
